In [1]:
from process_bigraph import Composite 

from biosimulators_processes import CORE 

Cannot register SimulariumSmoldynStep. Error:
**
No module named 'simulariumio'
**
Cannot register MongoDatabaseEmitter. Error:
**
No module named 'simulariumio'
**


In [2]:
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

doc = {
    'dFBA': {
        '_type': 'process',
        'address': 'local:dfba-process',
        'config': {
            'model': {
                'model_source': model_fp
            },
            'simulator': 'copasi',
            'start': 0,
            'stop': 10,
            'steps': 100
        },
        'inputs': {},
        'outputs': {
            'solution': ['solution_store']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'solution': 'tree[float]'
            }
        },
        'inputs': {
            'solution': ['solution_store']
        }
    }
}


spec = doc.copy()

In [ ]:
comp = Composite(
    config={'state': spec},
    core=CORE
)

In [ ]:
comp.run(3)

In [7]:
import numpy as np
import os 
from pathlib import Path
from tqdm import tqdm

from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import cobra
from cobra.io import load_model as load_cobra, read_sbml_model
from basico import * 

from biosimulators_processes.helpers import generate_reaction_mappings


# set up models
model_file = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

# set up cobra model
data_dir = Path(os.path.dirname(model_file))
path = data_dir / model_file.split('/')[-1]
cobrapy = read_sbml_model(str(path.resolve()))  # load_cobra('textbook')

# set up copasi model
copasi = load_model(model_fp)


Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction4 "translation of LacI">'
Missing upper flux bound se

In [ ]:
def add_dynamic_bounds(fba_model, utc_model, y):
    # 1. get reaction mappings 
    # 2. y = get_species(model=copasi).initial_concentration.values
    # 3. for each species in y, calculate max export (TODO: make this specific)
    biomass, glucose = y 
    glucose_max_import = -10 * glucose / (5 + glucose)
    fba_model.reactions.EX_glc__D_e.lower_bound = glucose_max_import


def dynamic_system(t, y):
    """Calculate the time derivative of external species."""

    biomass, glucose = y  # expand the boundary species

    # Calculate the specific exchanges fluxes at the given external concentrations.
    with cobrapy:
        add_dynamic_bounds(cobrapy, copasi, y)

        cobra.util.add_lp_feasibility(cobrapy)
        feasibility = cobra.util.fix_objective_as_constraint(cobrapy)
        lex_constraints = cobra.util.add_lexicographic_constraints(
            cobrapy, ['Biomass_Ecoli_core', 'EX_glc__D_e'], ['max', 'max'])

    # Since the calculated fluxes are specific rates, we multiply them by the
    # biomass concentration to get the bulk exchange rates.
    fluxes = lex_constraints.values
    fluxes *= biomass

    # This implementation is **not** efficient, so I display the current
    # simulation time using a progress bar.
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description('t = {:.3f}'.format(t))

    return fluxes

dynamic_system.pbar = None


def infeasible_event(t, y):
    """
    Determine solution feasibility.

    Avoiding infeasible solutions is handled by solve_ivp's built-in event detection.
    This function re-solves the LP to determine whether or not the solution is feasible
    (and if not, how far it is from feasibility). When the sign of this function changes
    from -epsilon to positive, we know the solution is no longer feasible.

    """

    with cobrapy:

        add_dynamic_bounds(cobrapy, y)

        cobra.util.add_lp_feasibility(cobrapy)
        feasibility = cobra.util.fix_objective_as_constraint(cobrapy)

    return feasibility - infeasible_event.epsilon

infeasible_event.epsilon = 1E-6
infeasible_event.direction = 1
infeasible_event.terminal = True

In [ ]:
def generate_reaction_mappings(output_names: list[str], reactions) -> list[dict]:
    mappings = []
    for reaction in reactions:
        for name in output_names:
            rxn = reaction.name.lower().split(" ")  # [r.lower() for r in list(reaction.values()).split(" ")]
            obs_name = name.split(" ")[0].lower()
            obs_type = name.split(" ")[-1]
            if obs_name in rxn:
                mapping = {}
                if "transcription" in rxn and obs_type == "mRNA":
                    mapping = {name: reaction.name}
                elif "translation" in rxn and obs_type == "protein":
                    mapping = {name: reaction.name}
                elif "degradation" in rxn:
                    if "transcripts" in rxn and obs_type == "mRNA":
                        mapping = {name: reaction.name}
                    elif "transcripts" not in rxn and obs_type == "protein":
                        mapping = {name: reaction.name}
                if mapping:
                    mappings.append(mapping)
    return mappings

In [ ]:
ts = np.linspace(0, 15, 100)  # Desired integration resolution and interval
y0 = [0.1, 10]

with tqdm() as pbar:
    dynamic_system.pbar = pbar

    sol = solve_ivp(
        fun=dynamic_system,
        events=[infeasible_event],
        t_span=(ts.min(), ts.max()),
        y0=y0,
        t_eval=ts,
        rtol=1e-6,
        atol=1e-8,
        method='BDF'
    )

In [ ]:
dir(sol)


In [ ]:
sol

In [ ]:
vars(sol)

In [ ]:
sol.sol

In [ ]:
sol.t

In [ ]:
type(sol)

In [ ]:
sol.status == 'optimal'
